# Crop Yield Prediction - ML Baseline

We use WOFOST crop growth indicators, weather variables, geographic information, soil data and remote sensing indicators to predict the yield.

## Google Colab Notes

**To run the script in Google Colab environment**
1. Download the data directory and save it somewhere convenient.
2. Open the notebook using Google Colaboratory.
3. Create a copy of the notebook for yourself.
4. Click connect on the right hand side of the bar below menu items. When you are connected to a machine, you will see a green tick mark and bars showing RAM and disk.
5. Click the folder icon on the left sidebar and click upload. Upload the data files you downloaded. Click *Ok* when you see a warning saying the files will be deleted after the session is disconnected.
6. Use *Runtime* -> *Run before* option to run all cells before **Set Configuration**.
7. Run the remaining cells except **Python Script Main**. The configuration subsection allows you to change configuration and rerun experiments.


## Install Spark

Install PySpark package. Package installation is required only in Google Colab.

In [ ]:
!pip install pyspark > /dev/null
!sudo apt update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install joblibspark > /dev/null

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

import pyspark
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

SparkContext.setSystemProperty('spark.executor.memory', '12g')
SparkContext.setSystemProperty('spark.driver.memory', '6g')
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

## Install MLBaseline package

Install cypml package from the test PyPi site. Note that 1.0.* has the baseline implementation. Later versions have additional improvements made to the baseline.


In [ ]:
! pip install -i https://test.pypi.org/simple/ cypml==1.0.7

Looking in indexes: https://test.pypi.org/simple/
     |████████████████████████████████| 61kB 3.3MB/s 


## Run Workflow

### Set Configuration


In [ ]:
from cypml.common import globals
from cypml.common.config import CYPConfiguration
from cypml.tests.test_util import TestUtil
from cypml.common.util import getLogFilename

cyp_config = CYPConfiguration()
run_tests = globals.run_tests

if (run_tests):
  test_util = TestUtil(spark)
  test_util.runAllTests()

my_config = {
      'crop_name' : 'sugarbeet',
      'season_crosses_calendar_year' : 'N',
      'country_code' : 'NL',
      'data_sources' : [ 'WOFOST', 'METEO_DAILY', 'SOIL', 'YIELD'],
      'data_path' : '.',
      'output_path' : '.',
      'nuts_level' : 'NUTS2',
      'use_yield_trend' : 'Y',
      'predict_yield_residuals' : 'N',
      'trend_windows' : [5, 7, 10],
      'use_centroids' : 'N',
      'use_remote_sensing' : 'Y',
      'early_season_prediction' : 'Y',
      'early_season_end_dekad' : -6,
      'save_features' : 'N',
      'use_saved_features' : 'N',
      'save_predictions' : 'N',
      'use_saved_predictions' : 'N',
      'compare_with_mcyfs' : 'Y',
      'debug_level' : 2,
}

cyp_config.updateConfiguration(my_config)
crop = cyp_config.getCropName()
country = cyp_config.getCountryCode()
nuts_level = cyp_config.getNUTSLevel()
debug_level = cyp_config.getDebugLevel()
use_saved_predictions = cyp_config.useSavedPredictions()
use_saved_features = cyp_config.useSavedFeatures()
use_yield_trend = cyp_config.useYieldTrend()
early_season_prediction = cyp_config.earlySeasonPrediction()
early_season_end = cyp_config.getEarlySeasonEndDekad()

print('##################')
print('# Configuration  #')
print('##################')
output_path = cyp_config.getOutputPath()
log_file = getLogFilename(crop, country, use_yield_trend,
                          early_season_prediction, early_season_end)
log_fh = open(output_path + '/' + log_file, 'w+')
cyp_config.printConfig(log_fh)

##################
# Configuration  #
##################

Current ML Baseline Configuration
--------------------------------
Crop name: sugarbeet
Crop ID: 6
Crop growing season crosses calendar year boundary: N
Country code (e.g. NL): NL
NUTS level for yield prediction: NUTS2
Input data sources: WOFOST, METEO_DAILY, SOIL, YIELD, REMOTE_SENSING
Estimate and use yield trend: Y
Predict yield residuals instead of full yield: N
Find optimal trend window: N
List of trend window lengths (number of years): 5, 7, 10
Use centroid coordinates and distance to coast: N
Use remote sensing data (FAPAR): Y
Predict yield early in the season: Y
Early season end dekad relative to harvest: -6
Path to all input data. Default is current directory.: .
Path to all output files. Default is current directory.: .
Save features to a CSV file: N
Use features from a CSV file: N
Save predictions to a CSV file: N
Use predictions from a CSV file: N
Compare predictions with MARS Crop Yield Forecasting System: Y
Debug l

### Load and Preprocess Data


In [ ]:
from cypml.workflow.data_loading import CYPDataLoader
from cypml.workflow.data_preprocessing import CYPDataPreprocessor
from cypml.tests.test_data_loading import TestDataLoader
from cypml.tests.test_data_preprocessing import TestDataPreprocessor
from cypml.run_workflow.run_data_preprocessing import preprocessData

if ((not use_saved_predictions) and
    (not use_saved_features)):

  print('#################')
  print('# Data Loading  #')
  print('#################')

  if (run_tests):
    test_loader = TestDataLoader(spark)
    test_loader.runAllTests()

  cyp_loader = CYPDataLoader(spark, cyp_config)
  data_dfs = cyp_loader.loadAllData()

  print('#######################')
  print('# Data Preprocessing  #')
  print('#######################')

  if (run_tests):
    test_preprocessor = TestDataPreprocessor(spark)
    test_preprocessor.runAllTests()

  cyp_preprocessor = CYPDataPreprocessor(spark, cyp_config)
  data_dfs = preprocessData(cyp_config, cyp_preprocessor, data_dfs)

#################
# Data Loading  #
#################
Data file name "./WOFOST_NUTS2_NL.csv"
Data file name "./METEO_DAILY_NUTS2_NL.csv"
Data file name "./SOIL_NUTS2_NL.csv"
Data file name "./YIELD_NUTS2_NL.csv"
Data file name "./REMOTE_SENSING_NUTS2_NL.csv"
Loaded data: WOFOST, METEO, SOIL, YIELD, REMOTE_SENSING


#######################
# Data Preprocessing  #
#######################
WOFOST data available for 12 region(s)
Season end information
+--------+-----+---------------+----------+
|IDREGION|FYEAR|PREV_SEASON_END|SEASON_END|
+--------+-----+---------------+----------+
|    NL11| 1979|              0|        36|
|    NL11| 1980|             36|        34|
|    NL11| 1981|             34|        34|
|    NL11| 1982|             34|        34|
|    NL11| 1983|             34|        36|
|    NL11| 1984|             36|        36|
|    NL11| 1985|             36|        36|
|    NL11| 1986|             36|        36|
|    NL11| 1987|             36|        36|
|    NL11| 1988|     

### Split Data into Training and Test Sets

In [ ]:
from cypml.tests.test_train_test_split import TestCustomTrainTestSplit
from cypml.run_workflow.run_train_test_split import splitDataIntoTrainingTestSets

if ((not use_saved_predictions) and
    (not use_saved_features)):

  print('###########################')
  print('# Training and Test Split #')
  print('###########################')

  if (run_tests):
    yield_df = data_dfs['YIELD']
    test_custom = TestCustomTrainTestSplit(yield_df)
    test_custom.runAllTests()

  prep_train_test_dfs, test_years = splitDataIntoTrainingTestSets(cyp_config, data_dfs, log_fh)

###########################
# Training and Test Split #
###########################

Test years: 2012, 2013, 2014, 2015, 2016, 2017, 2018



### Summarize Data

In [ ]:
from cypml.tests.test_data_summary import TestDataSummarizer
from cypml.workflow.data_summary import CYPDataSummarizer
from cypml.run_workflow.run_data_summary import summarizeData

if ((not use_saved_predictions) and
    (not use_saved_features)):

  print('#################')
  print('# Data Summary  #')
  print('#################')

  if (run_tests):
    test_summarizer = TestDataSummarizer(spark)
    test_summarizer.runAllTests()

  cyp_summarizer = CYPDataSummarizer(cyp_config)
  summary_dfs = summarizeData(cyp_config, cyp_summarizer, prep_train_test_dfs)

#################
# Data Summary  #
#################
Crop calender information based on WOFOST data
+--------+-------------+---------+----------+----------+---------------------+
|IDREGION|CAMPAIGN_YEAR|START_DVS|START_DVS1|START_DVS2|CAMPAIGN_EARLY_SEASON|
+--------+-------------+---------+----------+----------+---------------------+
|    NL11|         2011|       13|        19|        30|                   25|
|    NL12|         2011|       15|        20|        31|                   26|
|    NL13|         2011|       13|        19|        29|                   24|
|    NL21|         2011|       11|        17|        28|                   23|
|    NL22|         2011|       11|        17|        27|                   22|
|    NL23|         2011|       12|        17|        28|                   23|
|    NL31|         2011|       12|        17|        27|                   22|
|    NL32|         2011|       12|        17|        28|                   23|
|    NL33|         2011|      

### Create Features

In [ ]:
from cypml.workflow.feature_design import CYPFeaturizer
from cypml.run_workflow.run_feature_design import createFeatures
from cypml.tests.test_yield_trend import TestYieldTrendEstimator
from cypml.workflow.yield_trend import CYPYieldTrendEstimator
from cypml.run_workflow.run_trend_feature_design import createYieldTrendFeatures

if ((not use_saved_predictions) and
    (not use_saved_features)):

  print('###################')
  print('# Feature Design  #')
  print('###################')

  # WOFOST, Meteo and Remote Sensing Features
  cyp_featurizer = CYPFeaturizer(cyp_config)
  pd_feature_dfs = createFeatures(cyp_config, cyp_featurizer,
                                  prep_train_test_dfs, summary_dfs, log_fh)

  # yield trend features
  if (use_yield_trend):
    yield_train_df = prep_train_test_dfs['YIELD'][0]
    yield_test_df = prep_train_test_dfs['YIELD'][1]

    if (run_tests):
      test_yield_trend = TestYieldTrendEstimator(yield_train_df)
      test_yield_trend.runAllTests()

    cyp_trend_est = CYPYieldTrendEstimator(cyp_config)
    pd_yield_train_ft, pd_yield_test_ft = createYieldTrendFeatures(cyp_config, cyp_trend_est,
                                                                   yield_train_df, yield_test_df,
                                                                   test_years)
    pd_feature_dfs['YIELD_TREND'] = [pd_yield_train_ft, pd_yield_test_ft]

###################
# Feature Design  #
###################
Yield min year 1994


/content/spark-3.0.0-bin-hadoop2.7/python/pyspark/sql/pandas/conversion.py:88: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 0.15.1 must be installed; however, your version was 0.14.1.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)



 WOFOST Aggregate Features: Training
  IDREGION  FYEAR  maxWLIM_YBp2  ...  maxWLAIp4  avgRSMp2  avgRSMp4
0     NL42   1995       2615.32  ...       3.68     89.34     31.71
1     NL34   2000       3082.30  ...       4.60     91.75     78.17
2     NL13   2006       3007.10  ...       4.16     90.38     59.01
3     NL12   1999       2393.70  ...       5.82     94.75     72.24
4     NL22   2004       3378.76  ...       5.67     91.03     77.11

[5 rows x 11 columns]

 WOFOST Aggregate Features: Test
  IDREGION  FYEAR  maxWLIM_YBp2  ...  maxWLAIp4  avgRSMp2  avgRSMp4
0     NL13   2016       2175.01  ...       4.36     95.45     78.89
1     NL32   2014       2763.33  ...       4.26     93.49     61.01
2     NL21   2018       4368.62  ...       3.10     80.75     24.96
3     NL42   2015       3801.85  ...       4.16     82.57     44.99
4     NL33   2012       2129.41  ...       4.85    103.96     84.24

[5 rows x 11 columns]

 WOFOST Features for Extreme Conditions: Training
  IDREGION  FYE

/content/spark-3.0.0-bin-hadoop2.7/python/pyspark/sql/pandas/conversion.py:88: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 0.15.1 must be installed; however, your version was 0.14.1.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)



Yield Trend Features: Train
    IDREGION  FYEAR    YIELD-5  ...    YIELD-2    YIELD-1  YIELD_TREND
104     NL11   1999  51.000000  ...  55.700001  47.000000        50.01
105     NL11   2000  54.400002  ...  47.000000  60.299999        55.92
106     NL11   2001  52.000000  ...  60.299999  59.099998        60.46
107     NL11   2002  55.700001  ...  59.099998  54.400002        58.15
108     NL11   2003  47.000000  ...  54.400002  55.299999        58.43

[5 rows x 8 columns]
Total 156 rows

Yield Trend Features: Test
    IDREGION  FYEAR    YIELD-5  ...    YIELD-2    YIELD-1  YIELD_TREND
173     NL11   2012  65.599998  ...  71.199997  75.800003        77.87
174     NL11   2013  70.900002  ...  75.800003  75.199997        76.46
175     NL11   2014  74.800003  ...  75.199997  74.599998        75.40
176     NL11   2015  71.199997  ...  74.599998  86.800003        85.72
177     NL11   2016  75.800003  ...  86.800003  76.000000        81.28

[5 rows x 8 columns]
Total 84 rows


### Combine Features and Labels

In [ ]:
from cypml.run_workflow.combine_features import combineFeaturesLabels

if ((not use_saved_predictions) and
    (not use_saved_features)):

  join_cols = ['IDREGION', 'FYEAR']
  pd_train_df, pd_test_df = combineFeaturesLabels(cyp_config, sqlContext,
                                                  prep_train_test_dfs, pd_feature_dfs,
                                                  join_cols, log_fh)

/content/spark-3.0.0-bin-hadoop2.7/python/pyspark/sql/pandas/conversion.py:88: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 0.15.1 must be installed; however, your version was 0.14.1.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)



Combine Features and Labels
---------------------------
Yield min year 1994

Data size after including SOIL data: 
Train 12 rows.
Test 12 rows.

Data size after including WOFOST features: 
Train 216 rows.
Test 84 rows.

Data size after including METEO features: 
Train 216 rows.
Test 84 rows.

Data size after including REMOTE_SENSING features: 
Train 143 rows.
Test 77 rows.

Data size after including yield trend features: 
Train 143 rows.
Test 77 rows.

Data size after including yield (label) data: 
Train 143 rows.
Test 77 rows.


All Features and labels: Training
   IDREGION  FYEAR  SM_WHC  ...    YIELD-1  YIELD_TREND      YIELD
51     NL11   1999    0.22  ...  47.000000        50.01  60.299999
42     NL11   2000    0.22  ...  60.299999        55.92  59.099998
49     NL11   2001    0.22  ...  59.099998        60.46  54.400002
48     NL11   2002    0.22  ...  54.400002        58.15  55.299999
47     NL11   2003    0.22  ...  55.299999        58.43  59.200001

[5 rows x 58 columns]

All

### Apply Machine Learning using scikit learn


In [ ]:
from cypml.run_workflow.load_saved_features import loadSavedFeaturesLabels
from cypml.run_workflow.run_machine_learning import getMachineLearningPredictions
from cypml.run_workflow.run_machine_learning import saveMLPredictions

if ((not use_saved_predictions) and
    (use_saved_features)):
    pd_train_df, pd_test_df = loadSavedFeaturesLabels(cyp_config, spark)

if ((not use_saved_predictions)):
  print('\n###################################')
  print('# Machine Learning using sklearn  #')
  print('###################################')

  pd_ml_predictions = getMachineLearningPredictions(cyp_config, pd_train_df, pd_test_df, log_fh)
  save_predictions = cyp_config.savePredictions()
  if (save_predictions):
    saveMLPredictions(cyp_config, sqlContext, pd_ml_predictions)


###################################
# Machine Learning using sklearn  #
###################################

Training and Evaluation
-------------------------

Training Data Size: 143 rows
X cols: 54, Y cols: 4
IDREGION  FYEAR  SM_WHC  maxWLIM_YBp2  maxTWCp2  maxWLAIp2  maxWLIM_YBp4  maxWLIM_YSp4  maxTWCp4  maxWLAIp4  avgRSMp2  avgRSMp4  RSMp1gt1STD  RSMp1lt1STD  RSMp1gt2STD  RSMp1lt2STD  RSMp2gt1STD  RSMp2lt1STD  RSMp2gt2STD  RSMp2lt2STD  RSMp3gt1STD  RSMp3lt1STD  RSMp3gt2STD  RSMp3lt2STD  RSMp4gt1STD  RSMp4lt1STD  RSMp4gt2STD  RSMp4lt2STD  avgTAVGp0  avgPRECp0  avgCWBp0  avgTAVGp1  avgPRECp1  avgTAVGp2  avgCWBp2  avgPRECp3  avgCWBp4  TMINp1gt1STD  PRECp1gt1STD  TMINp1lt1STD  TMINp1gt2STD  PRECp1gt2STD  TMINp1lt2STD  PRECp3gt1STD  TMAXp3gt1STD  TMAXp3lt1STD  PRECp3gt2STD  TMAXp3gt2STD  TMAXp3lt2STD  avgFAPARp2  avgFAPARp4    YIELD-5    YIELD-4    YIELD-3    YIELD-2    YIELD-1  YIELD_TREND      YIELD
    NL11   1999    0.22       2919.89      3.51       3.19      14426.19       7144.9

### Compare Predictions with JRC Predictions

In [ ]:
from cypml.run_workflow.load_saved_predictions import loadSavedPredictions
from cypml.run_workflow.compare_with_mcyfs import comparePredictionsWithMCYFS

if (use_saved_predictions):
  pd_ml_predictions = loadSavedPredictions(cyp_config, spark)

compareWithMCYFS = cyp_config.compareWithMCYFS()
if (compareWithMCYFS):
  comparePredictionsWithMCYFS(sqlContext, cyp_config, pd_ml_predictions, log_fh)

log_fh.close()

##############
# Load Data  #
##############
Data file name "./WOFOST_NUTS2_NL.csv"
Data file name "./AREA_FRACTIONS_NUTS2_NL.csv"
Data file name "./AREA_FRACTIONS_NUTS1_NL.csv"
Data file name "./YIELD_NUTS0_NL.csv"
Data file name "./YIELD_PRED_MCYFS_NUTS0_NL.csv"
Loaded data: WOFOST, AREA_FRACTIONS, YIELD, YIELD_PRED_MCYFS


####################
# Preprocess Data  #
####################
NUTS0 Yield before preprocessing
+------+--------+-----+-----+
|  CROP|IDREGION|FYEAR|YIELD|
+------+--------+-----+-----+
|potato|      NL| 1971| 37.3|
|potato|      NL| 1972| 37.5|
|potato|      NL| 1973| 36.8|
|potato|      NL| 1974| 38.4|
|potato|      NL| 1975| 33.1|
|potato|      NL| 1976| 29.8|
|potato|      NL| 1977| 33.8|
|potato|      NL| 1978| 38.6|
|potato|      NL| 1979| 37.8|
|potato|      NL| 1980| 36.3|
+------+--------+-----+-----+
only showing top 10 rows

NUTS0 Yield after preprocessing
+--------+-----+-----+
|IDREGION|FYEAR|YIELD|
+--------+-----+-----+
|      NL| 2012| 78.9|
|     